In [1]:
import joblib 
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation , Flatten , Conv2D , MaxPooling2D ,Dense
import joblib

c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shubh\appd

train_data_path = "smoke/train/"
test_data_path = "smoke/test/"

In [2]:
train_data_path = "smoke/train/"
test_data_path = "smoke/test/"

In [3]:
train_data = []
test_data = []
categories = ["non smoky", "smoky"]
for category in categories:
    index = categories.index(category)
    path = os.path.join(train_data_path , category)
    for image in os.listdir(path=path):
        imag = cv2.imread(os.path.join(path , image) , 1)
        imag = cv2.resize(imag , (150,150))
        train_data.append([imag , index])
        
        


In [4]:
categories = ["non smoky", "smoky"]
for category in categories:
    index = categories.index(category)
    path = os.path.join(test_data_path , category)
    for image in os.listdir(path=path):
        imag = cv2.imread(os.path.join(path , image) , 1)
        imag = cv2.resize(imag , (150,150))
        print(imag.shape)
        test_data.append([imag , index])
        

(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)


In [5]:
x_test = []
y_test = []

for features , label in test_data:
    x_test.append(features)
    y_test.append(label)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = x_test/255.0

In [6]:
# print(categories.index(category))
# train_data[0]
x_data = []
y_data = []

for features , label in train_data:
    x_data.append(features)
    y_data.append(label)
x_data = np.array(x_data)
y_data = np.array(y_data)


In [7]:
x = x_data/255.0
y = y_data

# x[0]
x.shape

(605, 150, 150, 3)

In [8]:
import random
random.shuffle(train_data)
random.shuffle(test_data)

In [9]:
dense_layers = [0,1,2]
layer_sizes = [32,64,128]
conv_layers =  [ 1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer , layer_size , conv_layer , int(time.time()))
            tensorbrd = TensorBoard(log_dir= "logs\{}".format(name))
            model = Sequential()
            model.add(Conv2D(layer_size , (3,3) , input_shape = x.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size = (2,2)))
            
            
            for layer in range(conv_layer-1):
                model.add(Conv2D(layer_size , (3,3) ))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size = (2,2)))
            
            model.add(Flatten())
            
            for dense_l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss= "binary_crossentropy" ,
                         optimizer= "adam",
                         metrics= ["accuracy"], 
                         )

            model.fit(x , y ,batch_size=32 , validation_split=0.2 , epochs=10 , callbacks = [tensorbrd])
            model.save("{}".format(name))
            scores = model.evaluate(x_test , y_test)
            


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 484 samples, validate on 121 samples
Epoch 1/10
484/484 [==============================] - 6s 12ms/sample - loss: 4.1708 - acc: 0.5269 - val_loss: 2.6648 - val_acc: 0.0000e+00
Epoch 2/10
484/484 [==============================] - 2s 4ms/sample - loss: 0.9279 - acc: 0.6983 - val_loss: 0.5575 - val_acc: 0.8512
Epoch 3/10
484/484 [==============================] - 2s 4ms/sample - loss: 0.3931 - acc: 0.8058 - val_loss: 0.4179 - val_acc: 0.8760
Epoch 4/10
484/484 [==============================] - 2s 4ms/sample - loss: 0.3151 - acc: 0.8595 - val_loss: 0.7714 - val_acc: 0.7686
Epoch 5/10
484/484 [==============================] - 2s 4ms/sample - loss: 0.2165 - acc: 0.9277 - val_loss: 0.7199 - val_acc: 0.8017
Epoch 6/10
484/484 [==============================] - 2s 4ms/sample

484/484 [==============================] - 6s 12ms/sample - loss: 0.6071 - acc: 0.6260 - val_loss: 0.9365 - val_acc: 0.2479
Epoch 2/10
484/484 [==============================] - 5s 10ms/sample - loss: 0.4014 - acc: 0.8264 - val_loss: 0.7044 - val_acc: 0.7851
Epoch 3/10
484/484 [==============================] - 5s 10ms/sample - loss: 0.3381 - acc: 0.8554 - val_loss: 0.4023 - val_acc: 0.9174
Epoch 4/10
484/484 [==============================] - 5s 10ms/sample - loss: 0.2484 - acc: 0.9153 - val_loss: 0.4081 - val_acc: 0.9091
Epoch 5/10
484/484 [==============================] - 5s 10ms/sample - loss: 0.2067 - acc: 0.9112 - val_loss: 0.7289 - val_acc: 0.8099
Epoch 6/10
484/484 [==============================] - 6s 12ms/sample - loss: 0.1181 - acc: 0.9525 - val_loss: 0.8284 - val_acc: 0.7851
Epoch 7/10
484/484 [==============================] - 7s 14ms/sample - loss: 0.0984 - acc: 0.9669 - val_loss: 0.5663 - val_acc: 0.8926
Epoch 8/10
484/484 [==============================] - 7s 14ms/samp

Epoch 7/10
484/484 [==============================] - 3s 5ms/sample - loss: 0.0505 - acc: 0.9876 - val_loss: 0.8724 - val_acc: 0.8595
Epoch 8/10
484/484 [==============================] - 3s 5ms/sample - loss: 0.0129 - acc: 0.9979 - val_loss: 0.9097 - val_acc: 0.8430
Epoch 9/10
484/484 [==============================] - 3s 5ms/sample - loss: 0.0061 - acc: 1.0000 - val_loss: 0.8508 - val_acc: 0.8512
Epoch 10/10
42/42 [==============================] - 0s 2ms/sample - loss: 0.2239 - acc: 0.9762
Train on 484 samples, validate on 121 samples
Epoch 1/10
484/484 [==============================] - 3s 7ms/sample - loss: 0.5984 - acc: 0.6446 - val_loss: 1.6418 - val_acc: 0.0744
Epoch 2/10
484/484 [==============================] - 3s 5ms/sample - loss: 0.4078 - acc: 0.8471 - val_loss: 0.6276 - val_acc: 0.8430
Epoch 3/10
484/484 [==============================] - 3s 5ms/sample - loss: 0.2545 - acc: 0.8926 - val_loss: 2.1977 - val_acc: 0.3471
Epoch 4/10
484/484 [==============================] - 

ResourceExhaustedError: OOM when allocating tensor with shape[700928,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_21/kernel/Initializer/random_uniform/RandomUniform (defined at <ipython-input-9-f0777a549400>:24) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'dense_21/kernel/Initializer/random_uniform/RandomUniform':
  File "c:\users\shubh\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\shubh\appdata\local\programs\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\shubh\appdata\local\programs\python\python37\lib\